# Descrizione notebook
In questo notebook si fa utilizzo dello User-Based Collaborative Filtering per raccomandare una barzelletta ad un utente.

In [1]:
# import delle librerie necessarie
import numpy as np
import pandas as pd
import math

In [2]:
# lettura dei rating normalizzati dal relativo CSV
normalized_ratings_df = pd.read_csv('./data/normalized_ratings.csv')

# lettura dei rating non normalizzati dal relativo CSV (serve solo per calcolare la predizione, nella
# quale serve calcolare il rating medio dell'utente)
no_normalized_ratings_df = pd.read_csv('./data/no_normalized_ratings.csv')

# lettura delle barzellette dal relativo CSV
jokes_df = pd.read_csv('./data/jokes.csv')

# Similarità tra utenti: Coefficiente di Pearson

In [3]:
# funzione per trovare la similarità tra due utenti: Coefficiente di Pearson
def pearson_correlation_coefficient(x, y): 
    t1, t2, t3 = 0, 0, 0 
    for i, j in zip(x, y):
        if not(math.isnan(i)) and not(math.isnan(j)):
            t1+=i*j # numeratore
            t2+=i*i # denominatore_1
            t3+=j*j # denominatore_2
    return t1 / (np.sqrt(t2) * np.sqrt(t3))

# Predizione
Assegnazione degli score per le barzellette non ancora valutate dall'utente target.

In [4]:
def score_user_item_ubcf(item_id, neighbours_df, neighbour_user_similarity, active_user_mean_rating):
    item_rating = neighbours_df[item_id]
    t1, t2, score = 0, 0, 0
    for similarity, norm_rating in zip(neighbour_user_similarity, item_rating):
        t1+= norm_rating * similarity # numeratore
        t2+= similarity # denominatore
    if t2 !=0:
        score = active_user_mean_rating + t1 / t2
    return score

# Raccomandazione

In [5]:
def recommendation_ubcf(active_user_id, normalized_ratings_df, no_normalized_ratings_df, val):
    # si prende la riga relativa all'id dell'utente
    active_user = normalized_ratings_df[normalized_ratings_df['user_id'] == active_user_id]
    # della riga si considerano solo i rating espressi dall'utente
    active_user_rating = active_user.iloc[:, 2:]
    # si droppa altrimenti con lui uscirebbe similarità massima (= 1)
    normalized_ratings_df = normalized_ratings_df.drop(normalized_ratings_df[normalized_ratings_df['user_id'] == active_user_id].index)
    # vengono messi i rating dell'utente target in una lista,
    # ravel() esempio: [[1,2],[3,4]] = [1,2,3,4]
    active_user_rating_list = active_user_rating.values.ravel()
    
    # si calcola la similarità tra l'utente target e tutti gli altri utenti
    similarity = []
    for i in range(normalized_ratings_df.shape[0]):
          similarity.append([normalized_ratings_df.iloc[i, 0], pearson_correlation_coefficient(active_user_rating_list, normalized_ratings_df.iloc[i, 2:])])
    # si crea un dataframe
    similarity = pd.DataFrame(similarity, columns = ['user_id', 'similarity'])
    # si buttano i vicini che sono troppo poco simili
    similarity = similarity[similarity['similarity'] > 0.3]
        
    # si prendono le barzellette raccomandabili all'utente
    recommendation_columns = []
    for column in active_user_rating.columns:
        if math.isnan(active_user_rating[column].values[0]):
            recommendation_columns.append(column)
    
    # si calcola la media delle valutazioni dello user target in modo 
    # da avere una predizione più precisa (formula vista nel corso)
    active_user_raw_ratings = no_normalized_ratings_df[no_normalized_ratings_df['user_id'] == active_user_id].iloc[:, 2:]
    active_user_mean_rating = np.mean(active_user_raw_ratings.drop(recommendation_columns, axis = 1).values)
    
    # si prendono i rating dei neighbor dell'utente target
    neighbours_df = normalized_ratings_df[normalized_ratings_df['user_id'].isin(similarity["user_id"])]
    # si selezionano solo le barzellette che si volgiono raccomandare all'utente (quelle che non ha valutato)
    neighbours_df = neighbours_df[['user_id'] + recommendation_columns]
    # raccomandazione della barzelletta con la predizione più alta
    top_score = -np.inf
    joke_to_suggest = ''
    final = []
    # per ogni barzellette che si potrebbe raccomandare si calcola lo score
    for column in recommendation_columns:
        # si devono prendere i 30 vicini più simili che hanno valutato la barzelletta che si vuole raccomandare
        app = neighbours_df[neighbours_df[column].notna()][["user_id", column]]
        # si fa un join con la matrice di similarità sulla base dello user_id
        app = pd.merge(app, similarity, on = "user_id")
        # si ordina in ordine decrescente per la similarità e si prendono i primi 30 elementi
        app = app.sort_values(by=['similarity'], ascending=False).head(100)
        # si predice la valutazione dell'utente in base ai suoi vicini
        score = score_user_item_ubcf(column, app, app["similarity"], active_user_mean_rating)
        if score > top_score:
            top_score = score
            joke_to_suggest = column
        app_2 = {}
        app_2["user_id"] = active_user_id
        app_2[column] = score
        final.append(app_2)
        
    if (not(val)):
        print("L'utente target è: ")
        print(active_user)
        print("La similarità tra l'utente target e gli altri utenti è: ")
        print(similarity)
        print("Le barzellette raccomandabili all'utente target sono:")
        print(recommendation_columns)
        print("Il rating medio espresso dall'utente è: " + str(active_user_mean_rating))
        for elem in final:
            print("per la barzelletta " + str(list(elem.keys())[1]) + " si è predetto un punteggio di: " + str(elem[list(elem.keys())[1]]))
        print('\nil valore più alto predetto è ' + str(top_score) + ' su 10')
        print('quindi la barzelletta da raccomandare per la quale si è predetto quel punteggio è: ', joke_to_suggest) 
        s = jokes_df[jokes_df["joke_id"] == int(joke_to_suggest[5:])]["joke"]
        for c in s: 
            print(c)

    return final

In [6]:
final = recommendation_ubcf(1001, normalized_ratings_df, no_normalized_ratings_df, False)

L'utente target è: 
      user_id  number_of_jokes_rated  joke_1  joke_2  joke_3  joke_4  \
1000     1001                     53     NaN     NaN     NaN     NaN   

        joke_5    joke_6    joke_7    joke_8  ...  joke_91  joke_92  joke_93  \
1000 -0.079057 -0.129057 -0.179057 -4.159057  ...      NaN      NaN      NaN   

      joke_94  joke_95  joke_96  joke_97  joke_98  joke_99  joke_100  
1000      NaN      NaN      NaN      NaN      NaN      NaN       NaN  

[1 rows x 102 columns]
La similarità tra l'utente target e gli altri utenti è: 
       user_id  similarity
8            9    0.471821
10          11    0.435376
12          13    0.423799
17          18    0.316562
18          19    0.307272
...        ...         ...
24970    24972    0.337646
24972    24974    0.313457
24976    24978    0.594536
24977    24979    0.393784
24980    24982    0.338746

[5289 rows x 2 columns]
Le barzellette raccomandabili all'utente target sono:
['joke_1', 'joke_2', 'joke_3', 'joke_4', 'joke_9